# Chatbot with Deep Learning

## Ask Chatbot to get answer based on previous reviews

In [52]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import regularizers
from tensorflow.keras.models import Sequential,load_model
from tensorflow.keras.layers import Dense,LSTM,GRU,Embedding,Dropout,Flatten
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from pickle import dump,load
import os
%matplotlib inline

In [59]:
#CONSTANTS
#EMBEDDING_DIM = 100
#VALIDATION_SPLIT = 0.3
SAMPLE_SIZE = 0.5
#MAX_SEQUENCE_LENGTH = 100
FILE_PATH = '../../6 - NPL files/yelp_training_set_review.json'
#GLOVE_DIR = '../../../Machine Learning documents/glove/'

#variables
train_text_len = 25 + 1

In [75]:
#data_file = pd.read_csv(FILE_PATH)
data_file = pd.read_json(FILE_PATH, lines=True)
data_file = data_file[data_file['stars'] == 1].sample(frac=SAMPLE_SIZE).copy()
all_text = data_file['text'].values
all_text = all_text.astype('str')
num_instances = all_text.shape[0]
all_text = '. '.join(all_text.flatten())

In [76]:
import spacy
nlp = spacy.load('en',disable=['parser', 'tagger','ner'])
nlp.max_length = num_instances * 1000

In [77]:
def separate_punc(doc_text):
    return [token.text.lower() for token in nlp(doc_text) if token.text not in '\n\n \n\n\n!"-#$%&()--.*+,-/:;<=>?@[\\]^_`{|}~\t\n ']

In [78]:
tokens = separate_punc(all_text)

In [41]:
text_sequences = []
for i in range(train_text_len,len(tokens)):
    seq = tokens[i-train_text_len:i]
    text_sequences.append(seq)

In [45]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text_sequences)
sequences = tokenizer.texts_to_sequences(text_sequences)
vocabulary_size = len(tokenizer.word_counts)

In [46]:
for i in sequences[0]:
    print(f'{i} : {tokenizer.index_word[i]}')

2 : i
964 : purchased
1504 : carpet
11 : for
321 : three
13076 : bedrooms
58 : from
14 : this
235 : home
2422 : depot
188 : location
13 : we
696 : moved
49 : all
8 : of
1 : the
1908 : furniture
44 : as
119 : well
44 : as
1 : the
2898 : built
10 : in
30301 : shelving
10 : in
48 : one


In [48]:
sequences = np.array(sequences) 
X = sequences[:,:-1]
y = sequences[:,-1]
y = to_categorical(y, num_classes=vocabulary_size+1)
seq_len = X.shape[1]

In [49]:
def create_model(vocabulary_size, seq_len):
    model = Sequential()
    model.add(Embedding(vocabulary_size, 25, input_length=seq_len))
    model.add(LSTM(150, return_sequences=True))
    model.add(LSTM(150))
    model.add(Dense(150, activation='relu'))

    model.add(Dense(vocabulary_size, activation='softmax'))
    
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
   
    model.summary()
    
    return model

In [50]:
model = create_model(vocabulary_size+1, seq_len)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 25, 25)            757550    
_________________________________________________________________
lstm (LSTM)                  (None, 25, 150)           105600    
_________________________________________________________________
lstm_1 (LSTM)                (None, 150)               180600    
_________________________________________________________________
dense (Dense)                (None, 150)               22650     
_________________________________________________________________
dense_1 (Dense)              (None, 30302)             4575602   
Total params: 5,642,002
Trainable params: 5,642,002
Non-trainable params: 0
_________________________________________________________________


In [79]:
model.fit(X, y, batch_size=128, epochs=300,verbose=1)
model.save('gen_text.h5')
dump(tokenizer,open('gen_text_tokenizer,’wb'))